In [1]:
import numpy as np
import pandas as pd
import requests
import json

In [52]:
steam_id_url = 'http://api.steampowered.com/ISteamApps/GetAppList/v2'
steam_details_api = 'http://store.steampowered.com/api/appdetails/?appids='
ratings_api = 'http://steamspy.com/api.php?request=appdetails&appid='

In [4]:
steam_ids = requests.get(steam_id_url)

In [5]:
steam_ids = json.loads(steam_ids.text)

In [6]:
df_steam_ids = pd.DataFrame.from_dict(steam_ids['applist']['apps'])

In [53]:
games = []
for id_app in df_steam_ids.appid.values[:5]:
    api_call = requests.get(steam_details_api + str(id_app))
    rates_call = requests.get(ratings_api + str(id_app))
    rates = json.loads(rates_call.text)
    app_desc = json.loads(api_call.text)
    game = app_desc[app_desc.keys()[0]]
    if game['success']:
        game_info = game['data']
        if game_info['type'] == "game":
            cats = [cat['description'] for cat in game_info['categories']]
            cats = ' '.join(cats)
            genres = [genre['description'] for genre in game_info['genres']]
            genres = ' '.join(genres)
            games.append((game_info['steam_appid'], game_info['name'],
                          game_info['detailed_description'],
                          game_info['short_description'], 
                          game_info['about_the_game'],
                          game_info['developers'],
                          game_info['publishers'],
                          cats, genres,
                          game_info['recommendations']['total'],
                          rates['score_rank']))

df_cols = ['id', 'name',
           'f_desc', 's_desc','about',
           'dev' , 'pub', 'cat',
           'genres', 'nb_rec', 'score']
df_info = pd.DataFrame.from_records(games, columns=df_cols)       

Multi-player Valve Anti-Cheat enabled
Action
Multi-player Valve Anti-Cheat enabled
Action


In [54]:
df_info.head()

,id,name,f_desc,s_desc,about,dev,pub,cat,genres,nb_rec,score
0,10,Counter-Strike,Play the world's number 1 online action game. ...,,Play the world's number 1 online action game. ...,[Valve],[Valve],Multi-player Valve Anti-Cheat enabled,Action,93917,97
1,20,Team Fortress Classic,One of the most popular online action games of...,,One of the most popular online action games of...,[Valve],[Valve],Multi-player Valve Anti-Cheat enabled,Action,3020,53
